In [1]:
# Instala as bibliotecas necessárias
%%capture
!pip install --upgrade transformers bitsandbytes datasets peft torch
!pip install python-Levenshtein

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Carrega o modelo com fine-tuning armazenado do Hugging Face
model_name = "azmina/llama_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset
from huggingface_hub import login

login(token="HF TOKEN")  # Substitua pelo token de acesso da organização

dataset = load_dataset("belisards/ementas_anotadas") # Carrega o dataset armazenado no Hugging Face

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


README.md:   0%|          | 0.00/718 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/152k [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/19.9k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/112 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/168 [00:00<?, ? examples/s]

In [5]:
# Transforma o dataset em um df pandas para a inferência
test = dataset["test"].to_pandas()

In [6]:
# Checa os rótulos de tema presentes no df
test.label_tema.unique()

array(['genero', 'direitos sexuais e reprodutivos',
       'violencia contra a mulher', 'feminicidio', 'maternidade',
       'economia', 'dignidade sexual', 'direitos sociais', 'politica'],
      dtype=object)

In [7]:
# Seleciona apenas as colunas de interesse e printa o df para checagem
test = test[['text', 'label_tema']]
test

,text,label_tema
0,Proíbe o ensino de determinados conteúdos rela...,genero
1,"Susta os efeitos da Portaria nº 665, de 23 dem...",genero
2,"Altera a Lei nº 8.080, de 19 de setembro de 19...",direitos sexuais e reprodutivos
3,Estabelece a obrigatoriedade de submissão à mo...,violencia contra a mulher
4,"Altera o Decreto-Lei nº 2.848, de 7 de dezembr...",violencia contra a mulher
...,...,...
163,Institui o dia nacional de combate ao estupro.,dignidade sexual
164,Dispõe sobre a obrigatoriedade de afixação de ...,violencia contra a mulher
165,"Altera o Ccódigo Penal e a lei nº 8.072, de 19...",violencia contra a mulher
166,Altera a Lei nº 9.394 de 20 de dezembro de 199...,direitos sociais


In [8]:
import torch
import pandas as pd
import re
from Levenshtein import distance as levenshtein_distance
from tqdm import tqdm

# Definir os rótulos de temas permitidos para a inferência
allowed_labels = [
    "genero",
    "direitos sociais",
    "economia",
    "violencia contra a mulher",
    "maternidade",
    "direitos sexuais e reprodutivos",
    "dignidade sexual",
    "politica",
    "feminicidio"
]

# Função para normalizar o texto, remover pontuação e transformar tudo em minúsculas
def normalize(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

# Verifica a similaridade com os temas permitidos
def find_most_similar(prediction, labels, threshold=0.8):
    """Find the most similar label to the prediction."""
    normalized_prediction = normalize(prediction)
    best_match = None
    best_similarity = 0

    for label in labels:
        normalized_label = normalize(label)
        similarity = 1 - (levenshtein_distance(normalized_prediction, normalized_label) / max(len(normalized_prediction), len(normalized_label)))

        if similarity > best_similarity:
            best_similarity = similarity
            best_match = label

    # Retorna o tema com o melhor score de similaridade com os permitidos
    return best_match if best_similarity >= threshold else best_match

# Modelo para rodar na GPU (se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Template de instrução
instruction = """
A seguir temos o resumo de um projeto de lei. Defina o tema do projeto de lei entre os seguintes temas:
- genero
- direitos sociais,
- economia,
- violencia contra a mulher,
- maternidade,
- direitos sexuais e reprodutivos,
- dignidade sexual,
- politica,
- feminicidio

Responda apenas com o nome do tema correspondente, sem mais comentários.

### Projeto de lei:
{}

### Response:
"""

# Função para rodar a inferência no modelo
def run_inference(prompt):
    prompt_formatted = instruction.format(prompt)

    # Tokenizar o texto
    inputs = tokenizer(prompt_formatted, return_tensors="pt", truncation=True, padding=True).to(device)

    # Gerar a resposta
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    # Decodificar a saída
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extrair a resposta
    predicted_class = generated_text.split("### Response:")[-1].strip()

    # Verificar similaridade com os labels permitidos
    final_label = find_most_similar(predicted_class, allowed_labels)

    return final_label

# Aplicar a inferência em cada texto do DataFrame com tqdm para monitorar o progresso
test['predicted_label'] = [run_inference(text) for text in tqdm(test['text'], desc="Processing texts")]

print(test)

Processing texts: 100%|██████████| 168/168 [05:39<00:00,  2.02s/it]

                                                  text  \
0    Proíbe o ensino de determinados conteúdos rela...   
1    Susta os efeitos da Portaria nº 665, de 23 dem...   
2    Altera a Lei nº 8.080, de 19 de setembro de 19...   
3    Estabelece a obrigatoriedade de submissão à mo...   
4    Altera o Decreto-Lei nº 2.848, de 7 de dezembr...   
..                                                 ...   
163     Institui o dia nacional de combate ao estupro.   
164  Dispõe sobre a obrigatoriedade de afixação de ...   
165  Altera o Ccódigo Penal e a lei nº 8.072, de 19...   
166  Altera a Lei nº 9.394 de 20 de dezembro de 199...   
167  Altera a Lei nº 8.742, de 07 de dezembro de 19...   

                          label_tema            predicted_label  
0                             genero                     genero  
1                             genero                     genero  
2    direitos sexuais e reprodutivos           direitos sociais  
3          violencia contra a mulher  v

In [11]:
# Checa as labels originais do dataset
real_labels = test.label_tema.unique()
real_labels

array(['genero', 'direitos sexuais e reprodutivos',
       'violencia contra a mulher', 'feminicidio', 'maternidade',
       'economia', 'dignidade sexual', 'direitos sociais', 'politica'],
      dtype=object)

In [12]:
# Checa se as labels preditas estão de fato entre as permitidas
test["pred_labe_is_valid"] = test["predicted_label"].isin(real_labels)
test["pred_labe_is_valid"].value_counts(normalize=True)

,proportion
pred_labe_is_valid,
True,1.0


In [14]:
# Métricas da inferência
valid_test = test[test["pred_labe_is_valid"] == True]

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print(classification_report(valid_test["label_tema"], valid_test["predicted_label"]))

                                 precision    recall  f1-score   support

               dignidade sexual       0.89      0.47      0.62        17
direitos sexuais e reprodutivos       1.00      0.37      0.54        19
               direitos sociais       0.00      0.00      0.00        19
                       economia       0.32      0.57      0.41        14
                    feminicidio       0.00      0.00      0.00         5
                         genero       0.91      0.77      0.83        13
                    maternidade       0.52      0.63      0.57        19
                       politica       1.00      0.75      0.86         8
      violencia contra a mulher       0.67      0.96      0.79        54

                       accuracy                           0.61       168
                      macro avg       0.59      0.50      0.51       168
                   weighted avg       0.62      0.61      0.58       168



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
